In [8]:
!git clone https://github.com/pallets/jinja

fatal: destination path 'jinja' already exists and is not an empty directory.


In [2]:
!pip install pydriller

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 2.1 MB/s eta 0:00:00


In [14]:
from pydriller import Repository
import csv

Path=r"/content/jinja"

with open("commits_info.csv", "w", newline="", encoding="utf-8") as f:
    file=csv.writer(f)
    file.writerow(["Hash", "Message", "Hashes of parents", "Is a merge commit?", "List of modified files"])

    commits=Repository(Path).traverse_commits()
    for i in commits:
        commit=i.msg.lower()

        for j in commit.split():
            found=False;
            if j in ['fix', 'bug', 'issue', 'patch', 'resolve', 'error', 'correct', 'repair']:
                file.writerow([i.hash,i.msg,[k for k in i.parents],i.merge,[new_file.new_path for new_file in i.modified_files]])
                found=True
                break
            if found:
                break


In [15]:
import pandas as pd

In [16]:
df=pd.read_csv(r"/content/commits_info.csv")
name=df.columns
print(name)

Index(['Hash', 'Message', 'Hashes of parents', 'Is a merge commit?',
       'List of modified files'],
      dtype='object')


In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
from pydriller import Repository
import csv
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

Path=r"/content/jinja"
Task_c=r"/content/commits_info.csv"


# Loading bug_fixing_commit hashes:
print("Perfoming task-d !")
hashes={i["Hash"] for i in csv.DictReader(open(Task_c, encoding="utf-8"))}


print("Generating LLM_Inference (file name)")
print()

tokenizer=AutoTokenizer.from_pretrained("mamiksik/CommitPredictorT5")
model=AutoModelForSeq2SeqLM.from_pretrained("mamiksik/CommitPredictorT5")

print("By the use of LLM CommitPridictoreT5")
print()

def llm_commit(i: str)->str:
    if(i.strip()):
        input=tokenizer(i,return_tensors="pt", truncation=True, max_length=500)
        output=model.generate(**input, max_length=70)

        return tokenizer.decode(output[0], skip_special_tokens=True)
    else:
        return ""


print("Generating Rectifier message by the use of LLM model codet5-large")
print()

model_name_rectifier = "Salesforce/codet5-small"  # Or "Salesforce/codet5-large"
tokenizer_rectifier = AutoTokenizer.from_pretrained(model_name_rectifier)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model_rectifier = AutoModelForSeq2SeqLM.from_pretrained(model_name_rectifier).to(device)

def generate_rectified_message(diff_text: str, original_msg: str) -> str:
    """Generate rectified commit message using CodeT5."""
    if not diff_text.strip():
        return "[No diff to analyze]"
    try:
        prompt = f"Original commit message: {original_msg}\nCode changes:\n{diff_text}\n\nGenerate a detailed and precise commit message describing all changes:"

        inputs = tokenizer_rectifier(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=512
        ).to(device)

        outputs = model_rectifier.generate(
            **inputs,
            max_length=64,
            num_beams=4,
            early_stopping=True
        )
        return tokenizer_rectifier.decode(outputs[0], skip_special_tokens=True)
    except Exception as e:
        return f"[Error generating rectifier message: {e}]"
def rectifier(message: str, filename: str, i: str, llm: str)->str:
    msg=message.strip()
    sample_msgs=["fix bug", "fixed bugs", "bug fix", "minor fix", "fix errors"]
    if(len(msg.split())<3 or msg.lower() in sample_msgs) and llm:
        return llm

    if filename and "test" in filename.lower():
        return f"{llm} (test-related)" if llm else f"{msg} (test-related)"
    if filename and "parser" in filename.lower():
        return f"{llm} (parser-related)" if llm else f"{msg} (parser-related)"

    return msg

print("Final Step 5")
print()

with open("/content/drive/MyDrive/Lab2_Final.csv", "w", newline="", encoding="utf-8") as f:
    file=csv.writer(f)
    file.writerow(["Hash", "Message", "Filename", "Source Code (before)", "Source Code (current)", "Diff", "LLM Inference (fix type)", "Rectified Message"])

    for i in Repository(Path).traverse_commits():
        if i.hash not in hashes:
            continue

        for j in i.modified_files:
            File=j.new_path if j.new_path else ""
            difference=j.diff if j.diff else ""

            if not File.endswith(".py"):
                continue
            if not j.source_code_before or not j.source_code:
                continue
            if not difference.strip():
                continue

            llm=llm_commit(difference)
            rectified=rectifier(i.msg, File, difference, llm)

            file.writerow([i.hash, i.msg.strip(), File, j.source_code_before if j.source_code_before else "", j.source_code if j.source_code else "", difference, llm, rectified])

print("Lab2_Final.csv made Successfully...!")


Perfoming task-d !
Generating LLM_Inference (file name)

By the use of LLM CommitPridictoreT5

Generating Rectifier message by the use of LLM model codet5-large

cuda
Final Step 5

Lab2_Final.csv made Successfully...!
